<a href="https://colab.research.google.com/github/hugopedro-ds/Open-finance-recommender/blob/main/notebooks/recommendations_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup + leitura do dataset (anexo .txt)

In [ ]:
import ast, os
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

DATASET_PATH = "/content/01_DataSetOpenFinance.txt"

assert os.path.exists(DATASET_PATH), f"Arquivo não encontrado: {DATASET_PATH}"

def load_clientes_dict(path: str) -> dict:
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read().strip()

    raw = raw.lstrip("\ufeff").strip()  # remove BOM

    # remove "clientes =" se vier assim
    if raw.lower().startswith("clientes") and "=" in raw[:50]:
        raw = raw.split("=", 1)[1].strip()

    # remove ; final se existir
    if raw.endswith(";"):
        raw = raw[:-1].strip()

    data = ast.literal_eval(raw)
    if not isinstance(data, dict):
        raise ValueError("O dataset não está no formato esperado (dict).")
    return data

clientes = load_clientes_dict(DATASET_PATH)

# lista de todos os produtos (colunas)
all_products = sorted({p for prod_dict in clientes.values() for p in prod_dict.keys()})

print("✅ clientes definido:", len(clientes))
print("✅ produtos encontrados:", len(all_products))
first_client = next(iter(clientes))
print("✅ exemplo de cliente:", first_client)
print("✅ exemplo de itens desse cliente:", list(clientes[first_client].items())[:5])


✅ clientes definido: 4
✅ produtos encontrados: 6
✅ exemplo de cliente: Ana
✅ exemplo de itens desse cliente: [('Cartão de Crédito', 1), ('Conta Corrente', 2), ('Poupança', 3), ('Renda Fixa', 4), ('Crédito Pessoal', 5)]


Construir a matriz usuário × item (intensidade)

In [ ]:
df_ui = pd.DataFrame.from_dict(clientes, orient="index")

# (opcional) garantir ids como string (evita surpresas)
df_ui.index = df_ui.index.astype(str)

df_ui = df_ui.reindex(columns=all_products).fillna(0.0)
df_ui = df_ui.apply(pd.to_numeric, errors="coerce").fillna(0.0)

print("✅ Shape (users x items):", df_ui.shape)
display(df_ui.head())


✅ Shape (users x items): (4, 6)


,Cartão de Crédito,Conta Corrente,Crédito Pessoal,Poupança,Renda Fixa,Renda Variável
Ana,1,2,5.0,3,4.0,0.0
Marcos,2,3,0.0,4,5.0,0.6
Pedro,3,4,7.0,5,0.0,0.0
Claudia,4,5,0.0,6,0.0,0.0


Baseline: Popularidade (para comparação)

In [ ]:
# Baseline: popularidade (produtos mais "fortes" no dataset)
pop_score = df_ui.sum(axis=0).sort_values(ascending=False)  # soma intensidades por item
df_pop = pop_score.reset_index()
df_pop.columns = ["produto", "score_popularidade"]

display(df_pop.head(10))

def recommend_popularity(df_ui: pd.DataFrame, user: str, top_n: int = 3) -> pd.DataFrame:
    user = str(user)
    user_vec = df_ui.loc[user]
    owned = set(user_vec[user_vec > 0].index)

    candidates = df_pop[~df_pop["produto"].isin(owned)].head(top_n).copy()
    candidates["cliente"] = user
    candidates["porque"] = "Baseline: itens mais populares no conjunto (soma de intensidades)"
    candidates = candidates.rename(columns={"produto": "recomendacao", "score_popularidade": "score_estimado"})
    return candidates[["cliente", "recomendacao", "score_estimado", "porque"]].reset_index(drop=True)


,produto,score_popularidade
0,Poupança,18.0
1,Conta Corrente,14.0
2,Crédito Pessoal,12.0
3,Cartão de Crédito,10.0
4,Renda Fixa,9.0
5,Renda Variável,0.6


Similaridade item-item (cosseno)

In [ ]:
items = df_ui.columns.tolist()

sim = cosine_similarity(df_ui.T)  # itens x itens
df_sim = pd.DataFrame(sim, index=items, columns=items)

display(df_sim.round(3).head())



,Cartão de Crédito,Conta Corrente,Crédito Pessoal,Poupança,Renda Fixa,Renda Variável
Cartão de Crédito,1.000,0.994,0.552,0.984,0.399,0.365
Conta Corrente,0.994,1.000,0.601,0.998,0.489,0.408
Crédito Pessoal,0.552,0.601,1.000,0.627,0.363,0.000
Poupança,0.984,0.998,0.627,1.000,0.539,0.431
Renda Fixa,0.399,0.489,0.363,0.539,1.000,0.781


Recomendador (Item-based Collaborative Filtering) + explicação do “porquê”

In [ ]:
def recommend_item_based(
    df_ui: pd.DataFrame,
    df_sim: pd.DataFrame,
    user: str,
    top_n: int = 3,
    min_score: float = None
) -> pd.DataFrame:
    user = str(user)
    user_vec = df_ui.loc[user]

    owned = user_vec[user_vec > 0].index.tolist()
    candidates = user_vec[user_vec == 0].index.tolist()

    # cold-start simples
    if len(owned) == 0:
        return pd.DataFrame([{
            "cliente": user,
            "recomendacao": "(cold-start) sem histórico",
            "score_estimado": np.nan,
            "porque": "Usuário sem interações; usar popularidade/regra/segmentação."
        }]).head(top_n)

    recs = []
    for item in candidates:
        sims = df_sim.loc[item, owned]
        ratings = user_vec[owned]

        denom = sims.abs().sum() + 1e-9
        score = float((sims * ratings).sum() / denom)

        contrib = (sims * ratings).sort_values(ascending=False)
        reasons = contrib.head(2).index.tolist()

        recs.append({
            "cliente": user,
            "recomendacao": item,
            "score_estimado": score,
            "porque": f"Recomendado por similaridade com: {', '.join(reasons)}"
        })

    df = pd.DataFrame(recs).sort_values("score_estimado", ascending=False)

    if min_score is not None:
        df = df[df["score_estimado"] >= min_score]

    return df.head(top_n).reset_index(drop=True)



Gerar recomendações Top-N para todos os clientes (resultado para o slide)

In [ ]:
all_recs = []
for u in df_ui.index:
    all_recs.append(recommend_item_based(df_ui, df_sim, u, top_n=3))

df_recs = pd.concat(all_recs, ignore_index=True)
display(df_recs.head(20))



,cliente,recomendacao,score_estimado,porque
0,Ana,Renda Variável,2.819865,"Recomendado por similaridade com: Renda Fixa, ..."
1,Marcos,Crédito Pessoal,3.373872,"Recomendado por similaridade com: Poupança, Re..."
2,Pedro,Renda Fixa,4.686598,"Recomendado por similaridade com: Poupança, Cr..."
3,Pedro,Renda Variável,4.054936,"Recomendado por similaridade com: Poupança, Co..."
4,Claudia,Renda Fixa,5.097915,"Recomendado por similaridade com: Poupança, Co..."
5,Claudia,Renda Variável,5.054936,"Recomendado por similaridade com: Poupança, Co..."
6,Claudia,Crédito Pessoal,5.042111,"Recomendado por similaridade com: Poupança, Co..."


Protótipo “materializado” (impressão tipo UI / cenário fictício)

In [ ]:
def print_recs_as_ui(df_recs: pd.DataFrame):
    for user in df_recs["cliente"].astype(str).unique():
        print("="*70)
        print(f"RECOMENDAÇÕES PARA: {user}")
        sub = df_recs[df_recs["cliente"].astype(str) == user]
        for _, row in sub.iterrows():
            print(f"- {row['recomendacao']} | score={row['score_estimado'] if pd.notna(row['score_estimado']) else 'NA'}")
            print(f"  {row['porque']}")
        print()

print_recs_as_ui(df_recs)



RECOMENDAÇÕES PARA: Ana
- Renda Variável | score=2.819864602107294
  Recomendado por similaridade com: Renda Fixa, Poupança

RECOMENDAÇÕES PARA: Marcos
- Crédito Pessoal | score=3.373872407277934
  Recomendado por similaridade com: Poupança, Renda Fixa

RECOMENDAÇÕES PARA: Pedro
- Renda Fixa | score=4.686597604618653
  Recomendado por similaridade com: Poupança, Crédito Pessoal
- Renda Variável | score=4.054935828295562
  Recomendado por similaridade com: Poupança, Conta Corrente

RECOMENDAÇÕES PARA: Claudia
- Renda Fixa | score=5.097915088629339
  Recomendado por similaridade com: Poupança, Conta Corrente
- Renda Variável | score=5.054935827465499
  Recomendado por similaridade com: Poupança, Conta Corrente
- Crédito Pessoal | score=5.0421111621070756
  Recomendado por similaridade com: Poupança, Conta Corrente



Avaliação simples (Leave-One-Out Hit@K) — “cara de nota máxima”

In [ ]:
import random

def hit_rate_leave_one_out(df_ui, df_sim, k=3, seed=42):
    rng = random.Random(seed)
    hits = 0
    total = 0

    for user in df_ui.index.astype(str):
        user_vec = df_ui.loc[user]
        owned = user_vec[user_vec > 0].index.tolist()
        if len(owned) < 2:
            continue

        hidden = rng.choice(owned)

        df_tmp = df_ui.copy()
        df_tmp.loc[user, hidden] = 0.0

        recs = recommend_item_based(df_tmp, df_sim, user, top_n=k)
        recommended = recs["recomendacao"].tolist()

        hits += int(hidden in recommended)
        total += 1

    return hits / total if total else np.nan

for k in [1, 2, 3]:
    print(f"Hit@{k}: {hit_rate_leave_one_out(df_ui, df_sim, k=k):.2f}")


Hit@1: 0.00
Hit@2: 0.50
Hit@3: 0.75


Código para Hit@K: CF vs POP

In [ ]:
import random
import numpy as np

# --- 1) Baseline POP: recomenda top-N itens mais populares (exclui os já possuídos)
pop_score = df_ui.sum(axis=0).sort_values(ascending=False)
df_pop = pop_score.reset_index()
df_pop.columns = ["produto", "score_popularidade"]

def recommend_popularity(df_ui: pd.DataFrame, user: str, top_n: int = 3) -> pd.DataFrame:
    user = str(user)
    user_vec = df_ui.loc[user]
    owned = set(user_vec[user_vec > 0].index)

    candidates = df_pop[~df_pop["produto"].isin(owned)].head(top_n).copy()
    candidates = candidates.rename(columns={"produto": "recomendacao", "score_popularidade": "score_estimado"})
    candidates["cliente"] = user
    candidates["porque"] = "Baseline: itens mais populares no conjunto"
    return candidates[["cliente", "recomendacao", "score_estimado", "porque"]].reset_index(drop=True)

# --- 2) Avaliação Leave-One-Out genérica (serve para CF e POP)
def hit_rate_leave_one_out_model(df_ui: pd.DataFrame, recommend_fn, k: int = 3, seed: int = 42):
    rng = random.Random(seed)
    hits = 0
    total = 0

    for user in df_ui.index.astype(str):
        user_vec = df_ui.loc[user]
        owned = user_vec[user_vec > 0].index.tolist()
        if len(owned) < 2:
            continue

        hidden = rng.choice(owned)

        df_tmp = df_ui.copy()
        df_tmp.loc[user, hidden] = 0.0

        recs = recommend_fn(df_tmp, user, top_n=k)
        recommended = recs["recomendacao"].tolist()

        hits += int(hidden in recommended)
        total += 1

    return hits / total if total else np.nan

# --- 3) Wrappers
def rec_cf_wrapper(df_tmp, user, top_n=3):
    return recommend_item_based(df_tmp, df_sim, user, top_n=top_n)

def rec_pop_wrapper(df_tmp, user, top_n=3):
    return recommend_popularity(df_tmp, user, top_n=top_n)

# --- 4) Prints para o slide (CF vs POP)
for k in [1, 2, 3]:
    cf = hit_rate_leave_one_out_model(df_ui, rec_cf_wrapper, k=k, seed=42)
    pop = hit_rate_leave_one_out_model(df_ui, rec_pop_wrapper, k=k, seed=42)
    print(f"Hit@{k} | CF: {cf:.2f} | POP: {pop:.2f}")


Hit@1 | CF: 0.00 | POP: 0.50
Hit@2 | CF: 0.50 | POP: 1.00
Hit@3 | CF: 0.75 | POP: 1.00


Exportar tabelas para usar nos slides

In [ ]:
df_ui.to_csv("/content/matriz_usuario_item.csv", index=True)
df_sim.to_csv("/content/similaridade_item_item.csv", index=True)
df_recs.to_csv("/content/recomendacoes_top3.csv", index=False)

print("Arquivos gerados em /content:")
print("- matriz_usuario_item.csv")
print("- similaridade_item_item.csv")
print("- recomendacoes_top3.csv")

# (opcional) baixar no Colab
from google.colab import files
files.download("/content/recomendacoes_top3.csv")



Arquivos gerados em /content:
- matriz_usuario_item.csv
- similaridade_item_item.csv
- recomendacoes_top3.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>